In [1]:
from transformers import BertTokenizer, BertModel
from models.encoders.bert_like import BertLikeConfig
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')




e:\miniconda\miniconda3\envs\dspert\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = [
    {
        'tokens': ['John', 'Smith', 'works', 'at', 'HelloGoogle', '.'],
        'chunks': [('PER', 0, 2), ('ORG', 4, 5)],
    },
    {
        'tokens': ['Jane', 'Doe', 'is', 'a', 'doctor', '.', "xin chao","toi ten"],
        'chunks': [('PER', 0, 2)],
    }
]

In [ ]:
for token in data[0]['tokens']:
    out=tokenizer.encode(token)
    print(out)

In [5]:
example1 = config.exemplify(data[0])
example1

{'sub_tok_ids': tensor([ 101, 2198, 3044, 2573, 2012, 7592, 3995, 8649, 2571, 1012,  102]),
 'ori_indexes': tensor([0, 1, 2, 3, 4, 4, 4, 4, 5])}

In [13]:
tokenizer.decode([ 101, 2198, 3044, 2573, 2012, 7592, 3995, 8649, 2571, 1012,  102])

'[CLS] john smith works at hellogoogle. [SEP]'

In [6]:
example2 = config.exemplify(data[1])
example2

{'sub_tok_ids': tensor([  101,  4869, 18629,  2003,  1037,  3460,  1012,  8418,  2078, 22455,
          2000,  2072,  2702,   102]),
 'ori_indexes': tensor([0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 7, 7])}

In [7]:
batch = config.batchify([example1, example2])
batch

{'sub_tok_ids': tensor([[  101,  2198,  3044,  2573,  2012,  7592,  3995,  8649,  2571,  1012,
            102,     0,     0,     0],
         [  101,  4869, 18629,  2003,  1037,  3460,  1012,  8418,  2078, 22455,
           2000,  2072,  2702,   102]]),
 'sub_mask': tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
           True,  True,  True,  True]]),
 'ori_indexes': tensor([[ 0,  1,  2,  3,  4,  4,  4,  4,  5, -1, -1, -1],
         [ 0,  1,  2,  3,  4,  5,  6,  6,  6,  7,  7,  7]])}

In [18]:
config = BertLikeConfig(tokenizer=tokenizer, bert_like=bert_model, output_hidden_states=True)
embedder = config.instantiate()

In [22]:
len(batch['sub_mask'][0])

14

In [ ]:
config

In [42]:
bert_hidden, all_bert_hidden, sub_mask = embedder(**batch)
all_bert_hidden[0][0].shape

torch.Size([8, 768])

In [35]:
bert_hidden.shape

torch.Size([2, 8, 768])

In [13]:
import importlib
from models.encoders import bert_like, query_bert_like
from models.encoders import span_bert_like

importlib.reload(span_bert_like)

importlib.reload(bert_like)
importlib.reload(query_bert_like)



<module 'models.encoders.query_bert_like' from 'e:\\KLTN\\DSpERT2\\models\\encoders\\query_bert_like.py'>

In [20]:
query = query_bert_like.QueryBertLikeEncoder(bert_model.encoder, num_layers=2)

In [ ]:
test = [1,2,3,4,5]
test[-4:]

In [43]:

span_config = span_bert_like.SpanBertLikeConfig(
        bert_like=bert_model,
        max_span_size=4,
        init_agg_mode='scaled_dot_attention',
        init_drop_rate=0.2,
        share_weights_ext=True,
        share_weights_int=True,
        num_layers=2
    )
span_encoder = span_config.instantiate()

In [50]:
span_outputs = span_encoder(all_bert_hidden)
span_outputs[2].shape

torch.Size([2, 7, 768])

In [51]:
all_bert_hidden[0].shape

torch.Size([2, 8, 768])

In [ ]:
print(batch['ori_indexes'].shape)

In [ ]:
bert_hidden.shape

In [6]:
import torch
import importlib
from models.encoders import encode
importlib.reload(encode)
from models.encoders.encode import EncoderConfig

In [33]:
# Identity Encoder
config = EncoderConfig(arch='IDENTITY', in_dim=768, in_drop_rates=0.1, hid_drop_rate=0.0)
identity_encoder = config.instantiate()
embedded = torch.randn(2, 10, 768)
output = identity_encoder(embedded)
print(output.shape)  # [2, 10, 768]

# FFN Encoder
config = EncoderConfig(arch='FFN', in_dim=768, hid_dim=128, num_layers=2, in_drop_rates=0.1, hid_drop_rate=0.5)
ffn_encoder = config.instantiate()
output = ffn_encoder(embedded)
print(output.shape)  # [2, 10, 128]

# RNN Encoder (LSTM)
config = EncoderConfig(arch='LSTM', in_dim=768, hid_dim=256, num_layers=1, in_drop_rates=0.1, hid_drop_rate=0.5)
rnn_encoder = config.instantiate()
output = rnn_encoder(embedded)
print(output.shape)  # [2, 10, 256]

# RNN Encoder (GRU)
config = EncoderConfig(arch='GRU', in_dim=768, hid_dim=256, num_layers=2, in_drop_rates=0.0, hid_drop_rate=0.5)
gru_encoder = config.instantiate()
output = gru_encoder(embedded)
print(output.shape)  # [2, 10, 256]

torch.Size([2, 10, 768])
torch.Size([2, 10, 128])
torch.Size([2, 10, 256])
torch.Size([2, 10, 256])


In [3]:
from utils import seq_lens2mask
import torch
seq_lens2mask(torch.tensor([1,2,3]),5)

tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False]])

In [21]:
torch.tensor([[1,2,3]]).squeeze(1).shape

torch.Size([1, 3])

In [42]:
x = torch.randn(2, 3)
x

tensor([[ 0.5173, -1.3610,  0.7019],
        [-1.1862,  2.3196,  0.8788]])

In [41]:
y = torch.randn(3, 3)
y

tensor([[ 1.1596,  0.3546,  1.0000],
        [-1.1154, -1.8652, -1.4587],
        [-0.6398, -2.4668,  0.0357]])

In [43]:
torch.cat([x,y], dim=0)

tensor([[ 0.5173, -1.3610,  0.7019],
        [-1.1862,  2.3196,  0.8788],
        [ 1.1596,  0.3546,  1.0000],
        [-1.1154, -1.8652, -1.4587],
        [-0.6398, -2.4668,  0.0357]])

In [46]:
embedd = torch.nn.Embedding(2,3)

In [56]:
embedd(torch.tensor([1,1]))

tensor([[ 0.2587, -0.6512,  1.3804],
        [ 0.2587, -0.6512,  1.3804]], grad_fn=<EmbeddingBackward0>)

In [66]:
['helo'] + ["hi"]

['helo', 'hi']

In [10]:
model

SpecificSpanExtractor(
  (bert_like): BertLikeEmbedder(
    (bert_like): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_featur

In [ ]:
import logging
logging.basicConfig(filename='example.log', filemode='a', level=logging.INFO,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from typing import List, Dict
import torch.optim as optim

from models.specific_span_extractor import SpecificSpanExtractorConfig, SpecificSpanExtractor
from models.decoders.specific_span_classification import SpecificSpanClsDecoderConfig
from models.encoders.encode import EncoderConfig
from utils import seq_lens2mask
from models.encoders.bert_like import BertLikeConfig
from models.encoders.span_bert_like import SpanBertLikeConfig

# data = [
#     {'tokens': ['John', 'Smith', 'works', 'at', 'HelloGoogle', '.'], 'chunks': [('PER', 0, 2), ('ORG', 4, 5)]},
#     {'tokens': ['Jane', 'Doe', 'is', 'a', 'doctor', '.', 'xin chao', 'toi ten'], 'chunks': [('PER', 0, 2)]}
# ]
from utils import read_json
test_data = read_json("data/processed_data/phoner_covid19/test.json")


def train(model, dataloader, optimizer, device='cpu', num_epochs=5):
    model.train()
    model.to(device)
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            
            # Chuyển batch sang device
            batch['bert_like']['sub_tok_ids'] = batch['bert_like']['sub_tok_ids'].to(device)
            batch['bert_like']['sub_mask'] = batch['bert_like']['sub_mask'].to(device)
            batch['bert_like']['ori_indexes'] = batch['bert_like']['ori_indexes'].to(device)
            batch['seq_lens'] = batch['seq_lens'].to(device)
            batch['mask'] = batch['mask'].to(device)
            for obj in batch['boundaries_objs']:
                obj['label_ids'] = obj['label_ids'].to(device)
            
            # Tính loss
            losses = model.forward(batch)
            loss = losses.mean()
            
            # Lan truyền ngược và cập nhật tham số
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

def evaluate(model, dataloader, device='cpu'):
    model.eval()
    model.to(device)
    
    total_loss = 0
    all_chunks = []
    all_gold_chunks = [entry['chunks'] for entry in data]
    
    with torch.no_grad():
        for batch in dataloader:
            # Chuyển batch sang device
            batch['bert_like']['sub_tok_ids'] = batch['bert_like']['sub_tok_ids'].to(device)
            batch['bert_like']['sub_mask'] = batch['bert_like']['sub_mask'].to(device)
            batch['bert_like']['ori_indexes'] = batch['bert_like']['ori_indexes'].to(device)
            batch['seq_lens'] = batch['seq_lens'].to(device)
            batch['mask'] = batch['mask'].to(device)
            for obj in batch['boundaries_objs']:
                obj['label_ids'] = obj['label_ids'].to(device)
            
            # Tính loss
            losses = model.forward(batch)
            total_loss += losses.mean().item()
            
            # Dự đoán chunks
            chunks = model.decode(batch)
            all_chunks.extend(chunks)
    
    avg_loss = total_loss / len(dataloader)
    print(f"Evaluation Loss: {avg_loss:.4f}")
    print("Predicted chunks:", all_chunks)
    print("Gold chunks:", all_gold_chunks)
    
    # Tính precision, recall, F1 (micro-average đơn giản)
    n_gold = sum(len(chunks) for chunks in all_gold_chunks)
    n_pred = sum(len(chunks) for chunks in all_chunks)
    n_true_positive = sum(len(set(g) & set(p)) for g, p in zip(all_gold_chunks, all_chunks))
    precision = n_true_positive / n_pred if n_pred > 0 else 0
    recall = n_true_positive / n_gold if n_gold > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
    return avg_loss, precision, recall, f1

def inference(model, dataloader, device='cpu'):
    model.eval()
    model.to(device)
    
    all_chunks = []
    with torch.no_grad():
        for batch in dataloader:
            # Chuyển batch sang device
            batch['bert_like']['sub_tok_ids'] = batch['bert_like']['sub_tok_ids'].to(device)
            batch['bert_like']['sub_mask'] = batch['bert_like']['sub_mask'].to(device)
            batch['bert_like']['ori_indexes'] = batch['bert_like']['ori_indexes'].to(device)
            batch['seq_lens'] = batch['seq_lens'].to(device)
            batch['mask'] = batch['mask'].to(device)
            for obj in batch['boundaries_objs']:
                obj['label_ids'] = obj['label_ids'].to(device)
            
            # Dự đoán chunks
            chunks = model.decode(batch)
            all_chunks.extend(chunks)
    
    print("Inference chunks:", all_chunks)
    return all_chunks

def main():
    # Thiết lập mô hình và dataloader
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

    bert_like_config = BertLikeConfig(tokenizer=tokenizer, bert_like=bert_model, freeze=False)
    span_bert_like_config = SpanBertLikeConfig(bert_like=bert_model, freeze=False, share_weights_ext=True, share_weights_int=True)
    decoder_config = SpecificSpanClsDecoderConfig(max_span_size=5)
    extractor_config = SpecificSpanExtractorConfig(
        decoder=decoder_config,
        bert_like=bert_like_config,
        span_bert_like=span_bert_like_config
    )

    dataset = Dataset(test_data, extractor_config, training=True)
    dataset.build_vocabs_and_dims(test_data)
    dataloader = DataLoader(dataset, batch_size=2, collate_fn=dataset.collate)

    model = extractor_config.instantiate()

    # Đếm tham số
    count_parameters(model)

    # Thiết lập optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    # Huấn luyện
    print("\nTraining...")
    train(model, dataloader, optimizer, device=device, num_epochs=5)

    # Đánh giá
    print("\nEvaluating...")
    evaluate(model, dataloader, device=device)

    # Suy luận
    print("\nInference...")
    inference(model, dataloader, device=device)

if __name__ == "__main__":
    main()

e:\miniconda\miniconda3\envs\dspert\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total parameters: 111,165,501
Trainable parameters: 111,165,501
cuda

Training...


KeyboardInterrupt: 

In [1]:
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [6]:

from dataset import Dataset
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

bert_like_config = BertLikeConfig(tokenizer=tokenizer, bert_like=bert_model, freeze=False)
span_bert_like_config = SpanBertLikeConfig(bert_like=bert_model, max_span_size=5, freeze=False, share_weights_ext=True, share_weights_int=True)
decoder_config = SpecificSpanClsDecoderConfig(idx2label=['PER', 'ORG'], max_span_size=5)
extractor_config = SpecificSpanExtractorConfig(
    decoder=decoder_config,
    bert_like=bert_like_config,
    span_bert_like=span_bert_like_config
)
data = [
    {'tokens': ['John', 'Smith', 'works', 'at', 'HelloGoogle', '.'], 'chunks': [('PER', 0, 2), ('ORG', 4, 5)]},
    {'tokens': ['Jane', 'Doe', 'is', 'a', 'doctor', '.', 'xin chao', 'toi ten'], 'chunks': [('PER', 0, 2)]}
]
dataset = Dataset(test_data, extractor_config, training=True)

In [7]:
print(dataset.summary)

The dataset consists 3,000 sequences
The average `tokens` length is 36.1
The maximum `tokens` length is 185
The dataset has 11,735 chunks of 10 types


In [ ]:
from utils import read_json
test_data = read_json("data/processed_data/phoner_covid19/test.json")

In [4]:
len(test_data)

3000

In [37]:
import sys

def check_pythonpath():
    """
    In ra tất cả các đường dẫn trong sys.path để kiểm tra Python Path.
    """
    print("Python Path (sys.path):")
    for path in sys.path:
        print(path)

check_pythonpath()

Python Path (sys.path):
e:\miniconda\miniconda3\envs\dspert\python311.zip
e:\miniconda\miniconda3\envs\dspert\DLLs
e:\miniconda\miniconda3\envs\dspert\Lib
e:\miniconda\miniconda3\envs\dspert

C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages
e:\miniconda\miniconda3\envs\dspert\Lib\site-packages
e:\miniconda\miniconda3\envs\dspert\Lib\site-packages\win32
e:\miniconda\miniconda3\envs\dspert\Lib\site-packages\win32\lib
e:\miniconda\miniconda3\envs\dspert\Lib\site-packages\Pythonwin


In [12]:
from dspert.utils import read_json
test_data = read_json("data/processed_data/phoner_covid19/test.json")
test_data[1]['tokens']

['Bác',
 'sĩ',
 'Trần',
 'Thanh',
 'Linh',
 ',',
 'từ',
 'Bệnh',
 'viện',
 'Chợ',
 'Rẫy',
 'chi',
 'viện',
 'phụ',
 'trách',
 'đơn',
 'nguyên',
 'hồi',
 'sức',
 'tích',
 'cực',
 ',',
 'cho',
 'biết',
 '"',
 'bệnh',
 'nhân',
 '416',
 '"',
 'vẫn',
 'đang',
 'duy',
 'trì',
 'ECMO',
 ',',
 'thở',
 'máy',
 ',',
 'hiện',
 'xơ',
 'phổi',
 'rất',
 'nhiều',
 '.']